# Case Study 3 - Recommendation Systems

Build a Metadata based recommendation system on Credits and keyword dataset

In [1]:

import pandas as pd
import numpy as np



### Reading  Metadata - credits dataset and keywords dataset

Also loading movies_metadata dataset to obtain the movie titles

In [3]:
cred_df = pd.read_csv('rec-data/credits.csv')
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
key_df = pd.read_csv('rec-data/keywords.csv')
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
df = pd.read_csv('rec-data/movies_metadata.csv')
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Preprocessing

In [7]:
key_df.shape, cred_df.shape, df.shape

((46419, 2), (45476, 3), (45466, 24))

In [8]:
key_df.nunique()

id          45432
keywords    25989
dtype: int64

In [10]:
cred_df.nunique()

cast    43019
crew    44669
id      45432
dtype: int64

In [11]:
df['id'].nunique()

45436

In [13]:
# dropping duplicates from the keywords
key_df.drop_duplicates(subset=['id'], inplace=True)

In [14]:
# dropping duplicates from the credits
cred_df.drop_duplicates(subset=['id'], inplace=True)

In [15]:
# dropping duplicates from the movies dataset
df.drop_duplicates(subset=['id'], inplace=True)

#### Changing and Merging Credits and Keywords

In [16]:
cred_df.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [17]:
# display cast column data 
cred_df.iloc[0]['cast']

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [18]:
# display crew column data 
cred_df.iloc[0]['crew']

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [19]:
# check for missing values
cred_df.isna().sum()

cast    0
crew    0
id      0
dtype: int64

There are no missing values.


It looks like cast and crew are strings and we need to convert it into a list so we use literal eval 

In [20]:
# Literal Evaluation
from ast import literal_eval

In [21]:
# Applying literal eval to cast
cred_df['cast'] = cred_df['cast'].apply(literal_eval)
cred_df['cast']

0        [{'cast_id': 14, 'character': 'Woody (voice)',...
1        [{'cast_id': 1, 'character': 'Alan Parrish', '...
2        [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3        [{'cast_id': 1, 'character': 'Savannah 'Vannah...
4        [{'cast_id': 1, 'character': 'George Banks', '...
                               ...                        
45471    [{'cast_id': 0, 'character': '', 'credit_id': ...
45472    [{'cast_id': 1002, 'character': 'Sister Angela...
45473    [{'cast_id': 6, 'character': 'Emily Shaw', 'cr...
45474    [{'cast_id': 2, 'character': '', 'credit_id': ...
45475                                                   []
Name: cast, Length: 45432, dtype: object

In [22]:
cred_df['cast'][0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [23]:
# Applying Lambda Function to get cast name alone
cred_df['cast'] = cred_df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
cred_df['cast']

0        [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1        [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2        [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3        [Whitney Houston, Angela Bassett, Loretta Devi...
4        [Steve Martin, Diane Keaton, Martin Short, Kim...
                               ...                        
45471          [Leila Hatami, Kourosh Tahami, Elham Korda]
45472    [Angel Aquino, Perry Dizon, Hazel Orencio, Joe...
45473    [Erika Eleniak, Adam Baldwin, Julie du Page, J...
45474    [Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...
45475                                                   []
Name: cast, Length: 45432, dtype: object

In [25]:
# Applying literal eval to crew
cred_df['crew'] = cred_df['crew'].apply(literal_eval)
cred_df['crew']

0        [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1        [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2        [{'credit_id': '52fe466a9251416c75077a89', 'de...
3        [{'credit_id': '52fe44779251416c91011acb', 'de...
4        [{'credit_id': '52fe44959251416c75039ed7', 'de...
                               ...                        
45471    [{'credit_id': '5894a97d925141426c00818c', 'de...
45472    [{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...
45473    [{'credit_id': '52fe4776c3a368484e0c8387', 'de...
45474    [{'credit_id': '533bccebc3a36844cf0011a7', 'de...
45475    [{'credit_id': '593e676c92514105b702e68e', 'de...
Name: crew, Length: 45432, dtype: object

In [26]:
cred_df['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [27]:
# Applying Lambda Function to get crew name alone
cred_df['crew'] = cred_df['crew'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
cred_df['crew']

0        [John Lasseter, Joss Whedon, Andrew Stanton, J...
1        [Larry J. Franco, Jonathan Hensleigh, James Ho...
2        [Howard Deutch, Mark Steven Johnson, Mark Stev...
3        [Forest Whitaker, Ronald Bass, Ronald Bass, Ez...
4        [Alan Silvestri, Elliot Davis, Nancy Meyers, N...
                               ...                        
45471    [Hamid Nematollah, Hamid Nematollah, Farshad M...
45472    [Lav Diaz, Lav Diaz, Dante Perez, Lav Diaz, La...
45473    [Mark L. Lester, C. Courtney Joyner, Jeffrey G...
45474              [Yakov Protazanov, Joseph N. Ermolieff]
45475                                      [Daisy Asquith]
Name: crew, Length: 45432, dtype: object

In [28]:
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [29]:
# check for missing values
key_df.isna().sum()

id          0
keywords    0
dtype: int64

There are no missing values. Again we can apply literal eval to keywords to make a list.

In [30]:
# Applying literal eval to keywords
key_df['keywords'] = key_df['keywords'].apply(literal_eval)
key_df['keywords']

0        [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1        [{'id': 10090, 'name': 'board game'}, {'id': 1...
2        [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3        [{'id': 818, 'name': 'based on novel'}, {'id':...
4        [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords, Length: 45432, dtype: object

In [31]:
key_df['keywords'][0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [32]:
# Applying Lambda Function to get keywords alone
key_df['keywords'] = key_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
key_df['keywords']

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
46414                                        [tragic love]
46415                                [artist, play, pinoy]
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords, Length: 45432, dtype: object

We can create a new df for the recommender system we need to create based on keywords and credits. Here we can include the following features  
1. cast
2. crew
3. keywords
4. title

In [42]:
df1 = df[['id','title']]
df1.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [34]:
# Since we can't pass a list as a variable we convert it into a string
cred_df['cast'] = cred_df.apply(lambda x: ', '.join(x['cast']),axis=1)

In [35]:
# Since we can't pass a list as a variable we convert it into a string
cred_df['crew'] = cred_df.apply(lambda x: ', '.join(x['crew']),axis=1)

In [36]:
# Since we can't pass a list as a variable we convert it into a string
key_df['keywords'] = key_df.apply(lambda x: ', '.join(x['keywords']),axis=1)

In [79]:
# creating the combined data frame
com_df = pd.merge(key_df,cred_df,on='id')
com_df.head()

,id,keywords,cast,crew
0,862,"jealousy, toy, boy, friendship, friends, rival...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","John Lasseter, Joss Whedon, Andrew Stanton, Jo..."
1,8844,"board game, disappearance, based on children's...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","Larry J. Franco, Jonathan Hensleigh, James Hor..."
2,15602,"fishing, best friend, duringcreditsstinger, ol...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Howard Deutch, Mark Steven Johnson, Mark Steve..."
3,31357,"based on novel, interracial relationship, sing...","Whitney Houston, Angela Bassett, Loretta Devin...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr..."
4,11862,"baby, midlife crisis, confidence, aging, daugh...","Steve Martin, Diane Keaton, Martin Short, Kimb...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na..."


In [80]:
com_df.dtypes

id           int64
keywords    object
cast        object
crew        object
dtype: object

In [81]:
df1.dtypes

id        int64
title    object
dtype: object

In [82]:
# to merge df1 and com_df on id they should have same dtype for id
def ToInt(x):
    try:
        return int(x)
    except:
        return 0

In [83]:
df1['id'] = df1['id'].apply(ToInt)
df1.dtypes

id        int64
title    object
dtype: object

In [84]:
# Final combined dataset
com_df = pd.merge(com_df,df1,on='id')
com_df.head(10)

,id,keywords,cast,crew,title
0,862,"jealousy, toy, boy, friendship, friends, rival...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...",Toy Story
1,8844,"board game, disappearance, based on children's...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","Larry J. Franco, Jonathan Hensleigh, James Hor...",Jumanji
2,15602,"fishing, best friend, duringcreditsstinger, ol...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Howard Deutch, Mark Steven Johnson, Mark Steve...",Grumpier Old Men
3,31357,"based on novel, interracial relationship, sing...","Whitney Houston, Angela Bassett, Loretta Devin...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...",Waiting to Exhale
4,11862,"baby, midlife crisis, confidence, aging, daugh...","Steve Martin, Diane Keaton, Martin Short, Kimb...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...",Father of the Bride Part II
5,949,"robbery, detective, bank, obsession, chase, sh...","Al Pacino, Robert De Niro, Val Kilmer, Jon Voi...","Michael Mann, Michael Mann, Art Linson, Michae...",Heat
6,11860,"paris, brother brother relationship, chauffeur...","Harrison Ford, Julia Ormond, Greg Kinnear, Ang...","Sydney Pollack, Barbara Benedek, Sydney Pollac...",Sabrina
7,45325,,"Jonathan Taylor Thomas, Brad Renfro, Rachael L...","David Loughery, Stephen Sommers, Peter Hewitt,...",Tom and Huck
8,9091,"terrorist, hostage, explosive, vice president","Jean-Claude Van Damme, Powers Boothe, Dorian H...","Peter Hyams, Karen Elise Baldwin, Gene Quintan...",Sudden Death
9,710,"cuba, falsely accused, secret identity, comput...","Pierce Brosnan, Sean Bean, Izabella Scorupco, ...","Martin Campbell, Ian Fleming, Jeffrey Caine, B...",GoldenEye


In [86]:
# combining cast, crew and keywords into one column
com_df['cck'] = com_df['cast'] + ', ' + com_df['crew'] + ', ' + com_df['keywords']
com_df['cck'][0]

"Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette, John Lasseter, Joss Whedon, Andrew Stanton, Joel Cohen, Alec Sokolow, Bonnie Arnold, Ed Catmull, Ralph Guggenheim, Steve Jobs, Lee Unkrich, Ralph Eggleston, Robert Gordon, Mary Helen Leasman, Kim Blanchette, Marilyn McCoppen, Randy Newman, Dale E. Grahn, Robin Cooper, John Lasseter, Pete Docter, Joe Ranft, Patsy Bouge, Norm DeCarlo, Ash Brannon, Randy Newman, Roman Figun, Don Davis, James Flamberg, Mary Beth Smith, Rick Mackay, Susan Bradley, William Reeves, Randy Newman, Andrew Stanton, Pete Docter, Gary Rydstrom, Karen Robert Jackson, Chris Montan, Rich Quade, Michael Berenstein, Colin Brady, Davey Crockett Feiten, Angie Glocka, Rex Grignon, Tom K. Gurney, Jimmy Hayward, Hal T. Hickel, Karen Kiser, Anthony B. LaMolinara, Guionne Leroy, Bud Luckey, Les Major, Glenn McQueen, Mark Oftedal, Jeff Pidgeon, J

In [87]:
com_df.drop(['cast','crew','keywords'], axis=1, inplace=True)
com_df.head()

,id,title,cck
0,862,Toy Story,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,8844,Jumanji,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,15602,Grumpier Old Men,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,31357,Waiting to Exhale,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,11862,Father of the Bride Part II,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [88]:
com_df.isnull().sum()

id       0
title    3
cck      0
dtype: int64

### Recommender System Using TFIDF

In [56]:
#TF IDF Vector 
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

In [65]:
#Construct the required TF-IDF matrix by applying the fit_transform method on the combine cck feature
tfidf_cck = tfidf.fit_transform(com_df['cck']) 

In [66]:
tfidf_cck.shape

(45432, 230819)

In [60]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

In [67]:
# Compute the cosine similarity matrix for cck
cosine_cck = linear_kernel(tfidf_cck, tfidf_cck)

In [68]:
cosine_cck

array([[1.        , 0.04574837, 0.02269389, ..., 0.02605897, 0.        ,
        0.        ],
       [0.04574837, 1.        , 0.        , ..., 0.02956852, 0.00716204,
        0.        ],
       [0.02269389, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02605897, 0.02956852, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.00716204, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [89]:
# with id (200) of the movie we are sorting based on similarity score for cck
[(cosine_cck[200][i], com_df['title'][i]) for i in cosine_cck[200].argsort()[:-20:-1]]

[(0.9999999999999992, 'To Wong Foo, Thanks for Everything! Julie Newmar'),
 (0.13784666780151544, 'Hurricane Bianca'),
 (0.1256291827622884, 'Palookaville'),
 (0.11800606447818546, 'Trick'),
 (0.11520675110836165, 'Futuresport'),
 (0.1137301472497459, 'The Truth About Jane'),
 (0.11307262780538853, 'Staying Together'),
 (0.11277043038476958, 'The Pallbearer'),
 (0.1077951949860595, 'Six Degrees of Separation'),
 (0.10757686184350775, 'Jungle Fever'),
 (0.10520203728743051, 'Up the Sandbox'),
 (0.10466044347853526, 'The Love Letter'),
 (0.10413822723512797, 'The Lightkeepers'),
 (0.10365313646016859, 'Paris is Burning'),
 (0.10297325861092048, 'Gizmo!'),
 (0.09731529867867467, 'Confirmation'),
 (0.09683039053213176, 'The Birdcage'),
 (0.09673559350889674, 'Home Movie'),
 (0.09636254262380944, 'At First Sight')]

In [90]:
# reverse mapping of movieids to get indices
indices = pd.Series(com_df.index, index=com_df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45427
Century of Birthing            45428
Betrayal                       45429
Satan Triumphant               45430
Queerama                       45431
Length: 45432, dtype: int64

In [91]:
# Function that takes in cck as input and gives recommendations 
def CreditKeyRecommender(title, cosine_sim=cosine_cck, df=com_df, indices=indices):

    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that cast member
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]


In [101]:
#Get recommendations for Iron Man
CreditKeyRecommender('Iron Man 2')

12584                                 Iron Man
20821                               Iron Man 3
17810                             The Avengers
26540                  Avengers: Age of Ultron
41218    Marvel Studios: Assembling a Universe
16966                                     Thor
17436       Captain America: The First Avenger
26549               Captain America: Civil War
26544                                  Ant-Man
31367                          The Jungle Book
Name: title, dtype: object

In [102]:
com_df['cck'].loc[com_df['title'] == 'Iron Man 2'].values

array(["Robert Downey Jr., Gwyneth Paltrow, Don Cheadle, Scarlett Johansson, Mickey Rourke, Sam Rockwell, Samuel L. Jackson, Clark Gregg, John Slattery, Garry Shandling, Paul Bettany, Kate Mara, Leslie Bibb, Jon Favreau, Christiane Amanpour, Philippe Bergeron, James Bethea, Michael Bruno, Kate Clark, Luminita Docan, François Duhamel, Larry Ellison, Adam Goldstein, Tim Guinee, Eric L. Haney, Ali Khan, Evgeniy Lazarev, Stan Lee, Isaiah Guyman Martin IV, Helena Mattsson, Keith Middlebrook, Anya Monzikova, Margy Moore, Olivia Munn, Elon Musk, Bill O'Reilly, Alejandro Patino, Davin Ransom, Karim Saleh, Brian Schaeffer, Phillipe Simon, Jack White, Melanie Brown, Krystal Ellsworth, Victoria Gracie, Gina Cantrell, Renee Herlocker, Jill Ann Pineda-Arnold, Sandy Colton, Annika Ihnat, Jenny Robinson, Lindsay Dennis, Jennifer D. Johnson, Lindsay Rosenberg, Hannah Douglass, Brooke Long, Rachele Brooke Smith, Nadine Ellis, Kylette Zamora, Ted Alderman, Donessa Alexander, Martin Andris, Cassity Atkin

In [106]:
com_df['cck'].loc[com_df['title'] == "Iron Man"].values

array(["Robert Downey Jr., Terrence Howard, Jeff Bridges, Shaun Toub, Gwyneth Paltrow, Faran Tahir, Paul Bettany, Leslie Bibb, Clark Gregg, Will Lyman, Jon Favreau, Samuel L. Jackson, Stan Lee, Tom Morello, Jim Cramer, Sayed Badreya, Bill Smitrovich, Peter Billingsley, Tim Guinee, Marco Khan, Daston Kalili, Ido Ezra, Kevin Foster, Garret Noël, Eileen Weisinger, Ahmed Ahmed, Fahim Fazli, Gerard Sanders, Tim Rigby, Russell Richardson, Nazanin Boniadi, Thomas Craig Plumer, Robert Berkman, Stacy Stas, Lauren Scyphers, Frank Nyi, Marvin Jordan, Donna Evans, Reid Harper, Summer Kylie Remington, Ava Rose Williams, Vladimir Kubr, Callie Croughwell, Javan Tahir, Sahar Bibiyan, Patrick O'Connell, Adam Harrington, Meera Simhan, Ben Newmark, Ricki Lander, Jeannine Kaspar, Sarah Cahill, Justin Rex, Zorianna Kit, Lana Kinnear, Nicole Lindeblad, Masha Lund, Gabrielle Tuite, Tim Griffin, Joshua Harto, Micah A. Hauptman, James Bethea, Jeffrey Ashkin, Russell Bobbitt, Vianessa Castaños, Mike Cochrane, C

In [107]:
com_df['cck'].loc[com_df['title'] == "The Avengers"].values

array(['Ralph Fiennes, Uma Thurman, Sean Connery, Patrick Macnee, Jim Broadbent, Fiona Shaw, Eddie Izzard, Eileen Atkins, John Wood, Carmen Ejogo, Keeley Hawes, Shaun Ryder, Nicholas Woodeson, Michael Godley, Richard Lumsden, Solly Assa, Jeremiah S. Chechik, Sydney Newman, Susan Ekins, Jerry Weintraub, Joel McNeely, Roger Pratt, Mick Audsley, Don MacPherson, london england, weather manipulation, wretch',
       "Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner, Tom Hiddleston, Clark Gregg, Cobie Smulders, Stellan Skarsgård, Samuel L. Jackson, Gwyneth Paltrow, Paul Bettany, Jenny Agutter, Walter Perez, Powers Boothe, Stan Lee, Arthur Darbinyan, Donald Li, Ashley Johnson, Harry Dean Stanton, Alicia Sixtos, Dieter Riesle, Jamie McShane, Jeff Wolfe, Jerzy Skolimowski, Maximiliano Hernández, Alexis Denisof, James Eckhouse, Enver Gjokaj, Damion Poitier, Tina Benko, Kirill Nikiforov, M'laah Kaur Singh, Rashmi Rustagi, Warren Kole, Jesse Garcia, 

We can see from the recommendations that there are many similarities in the combined credits and keywords list. 